In [156]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import SearchIO
from BCBio import GFF
import csv
import sqlite3
import time
from multiprocessing import Pool, cpu_count
import sys
from calc_icity_ import *
import ast
from collections import defaultdict
import subprocess
import numpy as np
import sys

### helper

In [140]:
# make multi.faa file with all dTnpB p90 clusters
def get_prot_sequence(pid):
    con = sqlite3.connect("80kprotein_stats.db")
    cur = con.cursor()
    cmd = "SELECT sequence FROM proteins WHERE pid = '%s'" % pid 
    #print(cmd)
    cur.execute(cmd)
    return str(cur.fetchone()[0]).replace("*", "")
    con.close()
def get_faas_protidlist(protidlist, outfile_path, minlen = 0):
    with open(outfile_path, "w") as outfile:
        for protid in protidlist:
            protseq = get_prot_sequence(protid)
            if len(protseq) > minlen:
                print(">" + protid, file=outfile)
                print(protseq, file=outfile)
def get_p90(bait_pid):
    conn = sqlite3.connect('genegraph.db')
    cursor = conn.cursor()
    perm_rep = None
    cmd_p = "SELECT p90 FROM clusters WHERE p100 = '%s'" % (bait_pid)
    cursor.execute(cmd_p)
    perm_rep = cursor.fetchone()[0]
    conn.close()
    return(perm_rep)
def get_p30(bait_pid):
    conn = sqlite3.connect('genegraph.db')
    cursor = conn.cursor()
    perm_rep = None
    cmd_p = "SELECT p30 FROM clusters WHERE p100 = '%s'" % (bait_pid)
    cursor.execute(cmd_p)
    perm_rep = cursor.fetchone()[0]
    conn.close()
    return(perm_rep)


In [137]:
# import multi fasta file
inpath = sys.argv[1]
outpath = sys.argv[2]

input_name = inpath.split('/')[-1].replace(".faa", "")

In [152]:
print(inpath)
print(outpath)

../tree/sigma70/dTnpB_p30s.faa
../tree/sigma70


### kalign

In [138]:
infile_k = inpath
outfile_k = "{}/{}_kalign.out.faa".format(outpath, input_name)

def kalign(infile, outfile):
    cmd = "kalign -i {} -o {}".format(infile, outfile)
    print(cmd)
    process = subprocess.Popen(cmd.split(' '))
    process.wait()
    

In [ ]:
kalign(infile_k, outfile_k)


## Build tree

In [ ]:
infile_tree = outfile_k
outfile_tree = outfile_k.replace("faa", "tree")
def fasttree(infile, outfile):
    cmd = "FastTree {} > {}".format(infile, outfile)
    print(cmd)
    time.sleep(2)
    os.system(cmd)
    #process = subprocess.Popen(cmd.split(' '))
fasttree(infile_tree, outfile_tree)



### scratch work - comment out later


#### make multi.faa file with all dTnpB p90 clusters

In [109]:
with open("../ggdb_dfs/dTnpB_anymut_target_analysis.filtered.tsv", "r") as infile:
    df_ids = pd.read_csv(infile, sep = '\t').iloc[[0,1,2,14],:2]
df_ids["baitp90s"] = df_ids["baitp100s"].apply(lambda x: str([get_p90(pid) for pid in ast.literal_eval(x)]))
df_ids["baitp30s"] = df_ids["baitp100s"].apply(lambda x: str([get_p30(pid) for pid in ast.literal_eval(x)]))

In [126]:
d_90, d_30 = defaultdict(set), defaultdict(set)

In [141]:
for i in range(df_ids.shape[0]):
    target_id = df_ids.iloc[i,0]
    bait_p90ids = ast.literal_eval(df_ids.iloc[i,2])
    for pid in bait_p90ids:
        d_90[pid].update([target_id])
    bait_p30ids = ast.literal_eval(df_ids.iloc[i,3])
    for pid in bait_p30ids:
        d_30[pid].update([target_id])

In [142]:
dTnpB_p90s = list(d_90.keys())
get_faas_protidlist(dTnpB_p90s, "../tree/sigma70/dTnpB_p90s.faa")

In [143]:
dTnpB_p30s = list(d_30.keys())
get_faas_protidlist(dTnpB_p30s, "../tree/sigma70/dTnpB_p30s.faa")

#### make multi.fna file with dTnpB locus


In [205]:
def get_dtnpb_fna(target_ids, outpath):
    with open(outpath, "w") as outfile:
        for target_id in target_ids:
            gff_path = "../annotate/annotategff/OUTPUT/{}.sorted.gff".format(target_id)
            fna_path = "../annotate/annotategff/INPUT/fna/{}.examples.fna".format(target_id)
            with open(fna_path) as seq_handle:
                seq_dict = SeqIO.to_dict(SeqIO.parse(seq_handle, "fasta"))
            with open(gff_path) as in_handle:
                for rec in GFF.parse(in_handle):
                    for i in range(len(rec.features)):
                        if rec.features[i].type == "CDS_bait":
                            bait_loc = rec.features[i].location
                            bait_id = rec.features[i].id
                            rec_id = rec.id
                            dTnpB_loci_seq = seq_dict[rec_id].seq[bait_loc.start + 0:bait_loc.end + 250]
                            rec_bait_id = "{}|{}".format(rec_id.split('|')[0], bait_id)
                            print(">" + rec_bait_id, file=outfile)
                            print(dTnpB_loci_seq, file=outfile)

In [212]:
target_ids = ["aa6af7e9289c3558d3", "23422406293d40c201", "c1050b21cc75640d51", "6bf6c4c7da68779d7a"]
outfile_dTnpB_fna = "{}/dTnpB_all_loci.fna".format(outpath)
get_dtnpb_fna(target_ids, outfile_dTnpB_fna)

In [217]:
# summary csv
with open(outfile_dTnpB_fna) as seq_handle, open("../tree/sigma70/annot_loci.csv", "w") as outfile:
    records = SeqIO.parse(seq_handle, "fasta")
    for record in records:
        header = record.id
        t30, b100 = header.split('|')
        tree_header = "{}|{}".format(t30, b100)
        b90, b30 = get_p90(b100), get_p30(b100)
        out = [tree_header, t30, b30, b90, b100]
        print(*out, sep = ",", file = outfile)

In [281]:
annot_loci_old.shape, annot_loci_metadata.shape, annot_loci_new.shape

((106, 6), (87, 4), (87, 9))

In [283]:
# make more advanced summary csv 
annot_loci_old = pd.read_csv("../tree/sigma70/annot_loci.csv", header = None)
annot_loci_metadata = pd.read_csv("../tree/sigma70/sigma70_hit_metadata.csv")
b100_t100_df = pd.read_csv("../ggdb_dfs/sig70_b100_t100.csv").loc[:, ["target_p100id", "bait_p100id"]]
annot_loci_old = annot_loci_old.rename({0: "header", 1:"sigma70_p30id", 2: "b30", 3: "b90", 4: "bait_p100id"}, axis = 1)
annot_loci_old = annot_loci_old.merge(b100_t100_df, on = "bait_p100id").rename({"target_p100id": "sigma70_p100id"}, axis = 1)
annot_loci_new = annot_loci_old.merge(annot_loci_metadata, on = "sigma70_p100id")
annot_loci_new.to_csv("../tree/sigma70/annot_loci_new.csv", index = None, header = None)
annot_loci_new.head()

,header,sigma70_p30id,b30,b90,bait_p100id,sigma70_p100id,assembly_id,IS91_copynumb,loci_per_genome
0,aa6af7e9289c3558d3|15ce911abbf167916e,aa6af7e9289c3558d3,15ce911abbf167916e,15ce911abbf167916e,15ce911abbf167916e,00e9f918a990abd36d,GCF_900115145,2,4
1,aa6af7e9289c3558d3|0344dc509e9dbaf433,aa6af7e9289c3558d3,0344dc509e9dbaf433,0344dc509e9dbaf433,0344dc509e9dbaf433,053e55efbf254f1c04,GCF_001951155,0,1
2,aa6af7e9289c3558d3|580b003666b8b31f03,aa6af7e9289c3558d3,544d19bbbb28945456,580b003666b8b31f03,580b003666b8b31f03,088272ff80d0904531,GCF_900102055,0,1
3,aa6af7e9289c3558d3|a994d0b22469f6e7c8,aa6af7e9289c3558d3,544d19bbbb28945456,a994d0b22469f6e7c8,a994d0b22469f6e7c8,10545f9f1029a3a265,GCF_900106795,2,1
4,aa6af7e9289c3558d3|93699abf295bb251ef,aa6af7e9289c3558d3,14bc66ab7912db1e07,93699abf295bb251ef,93699abf295bb251ef,165a9103de9a151c12,GCF_012910715,0,1


In [284]:
annot_loci_new

,header,sigma70_p30id,b30,b90,bait_p100id,sigma70_p100id,assembly_id,IS91_copynumb,loci_per_genome
0,aa6af7e9289c3558d3|15ce911abbf167916e,aa6af7e9289c3558d3,15ce911abbf167916e,15ce911abbf167916e,15ce911abbf167916e,00e9f918a990abd36d,GCF_900115145,2,4
1,aa6af7e9289c3558d3|0344dc509e9dbaf433,aa6af7e9289c3558d3,0344dc509e9dbaf433,0344dc509e9dbaf433,0344dc509e9dbaf433,053e55efbf254f1c04,GCF_001951155,0,1
2,aa6af7e9289c3558d3|580b003666b8b31f03,aa6af7e9289c3558d3,544d19bbbb28945456,580b003666b8b31f03,580b003666b8b31f03,088272ff80d0904531,GCF_900102055,0,1
3,aa6af7e9289c3558d3|a994d0b22469f6e7c8,aa6af7e9289c3558d3,544d19bbbb28945456,a994d0b22469f6e7c8,a994d0b22469f6e7c8,10545f9f1029a3a265,GCF_900106795,2,1
4,aa6af7e9289c3558d3|93699abf295bb251ef,aa6af7e9289c3558d3,14bc66ab7912db1e07,93699abf295bb251ef,93699abf295bb251ef,165a9103de9a151c12,GCF_012910715,0,1
...,...,...,...,...,...,...,...,...,...
82,6bf6c4c7da68779d7a|12a498aea19ce6ec45,6bf6c4c7da68779d7a,6e87489ba377373838,51f58a1d54bd53ec9d,12a498aea19ce6ec45,c18a3f3190b6fda7d3,GCF_014692525,0,1
83,6bf6c4c7da68779d7a|994f61b1870f322add,6bf6c4c7da68779d7a,55da22e58a862b21b0,be5a84c48d88fedfd8,994f61b1870f322add,4532b45c4e7edc67c0,GCF_900116225,1,1
84,6bf6c4c7da68779d7a|4fc224ec001602f033,6bf6c4c7da68779d7a,55da22e58a862b21b0,be5a84c48d88fedfd8,4fc224ec001602f033,cf1183adcd3348274f,GCF_011030405,1,2
85,6bf6c4c7da68779d7a|49ef6279f6c4803c04,6bf6c4c7da68779d7a,6e87489ba377373838,75cb24fd8f1b45b311,49ef6279f6c4803c04,dd5b5c6ad215be5e9d,GCF_011007365,0,1


#### make multi.fna file with wRNAs

In [219]:
def get_wrna_fna(target_ids, outpath, num_bases_tnpb_orf):
    with open(outpath, "w") as outfile:
        for target_id in target_ids:
            gff_path = "../annotate/annotategff/OUTPUT/{}.sorted.gff".format(target_id)
            fna_path = "../annotate/annotategff/INPUT/fna/{}.examples.fna".format(target_id)
            with open(fna_path) as seq_handle:
                seq_dict = SeqIO.to_dict(SeqIO.parse(seq_handle, "fasta"))
            with open(gff_path) as in_handle:
                for rec in GFF.parse(in_handle):
                    for i in range(len(rec.features)):
                        if rec.features[i].type == "CDS_bait":
                            bait_loc = rec.features[i].location
                            bait_id = rec.features[i].id
                            rec_id = rec.id
                            dTnpB_loci_seq = seq_dict[rec_id].seq[bait_loc.end - num_bases_tnpb_orf:bait_loc.end + 250]
                            rec_bait_id = "{}|{}".format(rec_id.split('|')[0], bait_id)
                            print(">" + rec_bait_id, file=outfile)
                            print(dTnpB_loci_seq, file=outfile)

In [220]:
outfile_wrna_fna_200 = "{}/wrna_all_loci.200.fna".format(outpath)
get_wrna_fna(target_ids, outfile_wrna_fna_200, 200)

In [222]:
outfile_wrna_fna_150 = "{}/wrna_all_loci.150.fna".format(outpath)
get_wrna_fna(target_ids, outfile_wrna_fna_150, 150)

In [221]:
outfile_wrna_fna_100 = "{}/wrna_all_loci.100.fna".format(outpath)
get_wrna_fna(target_ids, outfile_wrna_fna_100, 100)

In [223]:
outfile_wrna_fna_0 = "{}/wrna_all_loci.0.fna".format(outpath)
get_wrna_fna(target_ids, outfile_wrna_fna_0, 0)